In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
import os
import anndata
from sklearn.mixture import GaussianMixture
from fcsy import DataFrame
import matplotlib
from glob import glob
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['ps.fonttype']=42
import warnings
warnings.filterwarnings("ignore")
from igraph import InternalError

# scanpy settings
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=150, frameon=False, figsize=(4, 4)) 
sc._settings.ScanpyConfig.n_jobs=4 # useless

In [2]:
# readin the information table
info_path = '/Users/tan/ionctura-collab/data/Sample_info_reformat.xlsx'
sampleInfo = pd.read_excel(info_path, dtype={'Sample ID':str}, sheet_name=0)
selectInfo = sampleInfo

In [3]:
dataDir = '/Users/tan/ionctura-collab/data/Monocytes and Tregs_iOnctura'
nsample = 3000

select_columns = ['CD11c', 'CD123', 'CD127', 'CD137', 'CD14', 'CD141', 'CD147',
       'CD16', 'CD161', 'CD1c', 'CD20', 'CD22', 'CD24', 'CD25', 'CD26',
       'CD27', 'CD28', 'CD29', 'CD3', 'CD33', 'CD38', 'CD39', 'CD4', 'CD43',
       'CD45', 'CD45RA', 'CD49d', 'CD5', 'CD52', 'CD55', 'CD56', 'CD57',
       'CD64', 'CD7', 'CD71', 'CD81', 'CD85j', 'CD8a', 'CD9', 'CD95', 'CD99',
       'CX3CR1', 'HLA-DR', 'IgD', 'Siglec-8', 'TCRgd', 'pan-KIR']
drop_dic = {'pDC': [], 
            'B-cells': ['CD33', 'CD3', 'TCRgd', 'Siglec-8', 'CD14', 'CD141', 'CD4', 'pan-KIR'],
            'CD4 T-cells': ['IgD', 'CD1c', 'TCRgd', 'Siglec-8', 'CD20', 'CD14'],
            'CD8 T-cells': ['IgD', 'CD11c', 'CD1c', 'TCRgd', 'Siglec-8', 'CD20', 'CD14'],
            #'Eosinophils': ['IgD', 'CD57', 'CD25', 'TCRgd', 'CD14', 'pan-KIR'],
            'Monocytes': ['CD57', 'IgD', 'CD25', 'CD20', 'TCRgd', 'CD22', 'CD127', 'pan-KIR'], 
            #'Neutrophils': ['IgD', 'HLA-DR', 'CD57', 'CD25', 'CD22', 'TCRgd', 'CD123', 'CD161', 'pan-KIR'],
            #'NK cells': [],
            #'Lin Neg': [],
            #'gdT': [], 
            #'Plasmablasts': [],
            #'Basophils': [],
            'Tregs':[]
           }


# readin and merge the file according to "selectInfo" as an anndata object
sub_name = 'Monocytes'
all_data_path = '../adata/ion_EXP-21-DG3637_56_manual_gating_' + sub_name + '.h5ad'
if not os.path.exists(all_data_path):
    print('adata not found, load and preprocess raw data...')
    all_data_list = []
    all_label_list = []
    for i in range(len(selectInfo)):
        data_path = glob(dataDir + '/*/*'+ sub_name + '_' + str(selectInfo['Sample_ID'].iloc[i]) + '*.csv')[0]
        dataTmp = pd.read_csv(data_path)
        labelTmp = pd.DataFrame(np.repeat(sub_name, len(dataTmp.index)), columns = ['subpop'])
        labelTmp['Sample ID'] = np.repeat(str(selectInfo['Sample_ID'].iloc[i]), len(labelTmp.index))
        labelTmp['timepoint'] = np.repeat(str(selectInfo['timepoint'].iloc[i]), len(labelTmp.index))
        labelTmp['Subject ID'] = np.repeat(str(selectInfo['Subject_ID'].iloc[i]), len(labelTmp.index))
        labelTmp['dose'] = np.repeat(str(selectInfo['Cohort_and_dose'].iloc[i]), len(labelTmp.index))
        labelTmp['type'] = np.repeat(str(selectInfo['Tumor_type'].iloc[i]), len(labelTmp.index))
        labelTmp['tumor_group'] = np.repeat(str(selectInfo['Tumor_Groups'].iloc[i]), len(labelTmp.index))
        labelTmp['timepoint_dose'] = np.repeat(selectInfo['timepoint'].iloc[i] + 
                                                '_' + 
                                                str(selectInfo['Cohort_and_dose'].iloc[i]), len(labelTmp.index))
        labelTmp['batch'] = np.repeat(str(selectInfo['Expt_ID'].iloc[i]), len(labelTmp.index))
        if '4-1BB' in set(dataTmp.columns):
            dataTmp.rename(columns={"4-1BB": "CD137"}, inplace=True)
        # remove EQBeads and DNA channel # also remove the negative channels
        #dataTmp.drop(columns=drop_columns, inplace=True)
        dataTmp = dataTmp[select_columns]
        dataTmp = np.arcsinh(dataTmp/5) # already transformed in FlowJo
        all_data_list.append(dataTmp)
        all_label_list.append(labelTmp)

    all_data = pd.concat(all_data_list, ignore_index=True)
    all_label = pd.concat(all_label_list, ignore_index=True)
    adata = anndata.AnnData(all_data)
    adata.obs = all_label
    adata.raw = adata
    print('batch correction...')
    #sc.pp.combat(adata, key = 'batch', covariates = ['tumor_group', 'timepoint', 'dose'])
    sc.pp.combat(adata)
    #print('scaling...')
    sc.pp.scale(adata)
    os.makedirs('adata/', exist_ok=True)
    print('write to h5ad file...')
    adata.write(filename=all_data_path, compression = 'gzip')
    adata=None
    print('finished!')
print('loading...')
adata_all = sc.read_h5ad(filename = all_data_path)
print('finished!')

adata not found, load and preprocess raw data...
batch correction...


... storing 'subpop' as categorical
... storing 'Sample ID' as categorical
... storing 'timepoint' as categorical
... storing 'Subject ID' as categorical
... storing 'dose' as categorical
... storing 'type' as categorical
... storing 'tumor_group' as categorical
... storing 'timepoint_dose' as categorical
... storing 'batch' as categorical


write to h5ad file...
finished!
loading...
finished!


In [4]:
for tumor_group in ['Tumor Group 1', 'Tumor Group 2', 'Tumor Group 3']:
    #tumor_group='Tumor Group 1'
    # subsampling
    print('subsampling...')
    try:
        sample_index = adata_all.obs.groupby('Sample ID').apply(lambda x: x.sample(n=nsample, random_state=0) if x.shape[0]>=nsample else x).index.droplevel(level=0)        
        adata_sample = adata_all[sample_index]
    except ValueError as e: 
        # usually when no file has more cells than nsample and thus no subsampling at all.
        # so the results after apply will not be a multiplex index and the droplevel func will fail.
        print(e)
        adata_sample = adata_all

    drop_indicator = np.in1d(adata_sample.var_names, drop_dic[sub_name])
    adata = adata_sample[:,~drop_indicator]

    adata = adata[adata.obs['tumor_group'] == tumor_group] # merge by tumor group

    # filter out those density_groups that have too few cells
    density_groups = adata.obs['timepoint'].value_counts()[adata.obs['timepoint'].value_counts()>20].index.sort_values().tolist()
    adata = adata[adata.obs['timepoint'].isin(density_groups)]
    # skip
    # optimization
    #sc.settings.figdir='./optimization/'
    #for res in [0.3]:
    #    adata_opt = adata
    #    n_comps = min([adata_opt.n_obs, adata_opt.n_vars, 21])-1
    #    sc.tl.pca(adata_opt, svd_solver='arpack', n_comps=n_comps)
    #    sc.pp.neighbors(adata_opt, n_neighbors=10, n_pcs=n_comps)
    #    sc.tl.leiden(adata_opt, resolution=res)
    #    sc.tl.paga(adata_opt, groups='leiden')
    #    sc.pl.paga(adata_opt, color=['leiden'], threshold=0.1, show=False, 
    #               save='_' + sub_name + '_' + str(res) + '.pdf')
    #    adata_opt = []

    # save figures to a sub dir
    figpath='../figures_manual_gating/' + tumor_group + '/' + sub_name + '/'
    h5path='../PAGA_result_data_manual_gating/' + tumor_group + '/'
    os.makedirs(figpath, exist_ok=True)
    sc.settings.figdir=figpath
    os.makedirs(h5path, exist_ok=True)

    print('calculating PAGA...')
    #n_comps = min([adata.n_obs, adata.n_vars, 21])-1
    #sc.tl.pca(adata, svd_solver='arpack', n_comps=n_comps)
    sc.pp.neighbors(adata, n_neighbors=10)

    # paga process
    sc.tl.leiden(adata, resolution=0.3) 

    sc.tl.paga(adata, groups='leiden')
    if adata.obs['leiden'].nunique() < 2: # no paga can be done with only 1 group
        sc.tl.draw_graph(adata)
        sc.pl.draw_graph(adata, color=['timepoint', 'dose', 'leiden', 'batch', 'Subject ID', 'type'], show=False,
                     save='_' + sub_name + '.pdf')
        raise ValueError('no paga can be done with only 1 group')
    try:
        sc.pl.paga(adata, color=['leiden'], threshold=0.1, show=False, 
                   save='_' + sub_name + '.pdf')
    except InternalError as e: # maybe there're too little cells
        print(e)
        sc.pl.paga(adata, color=['leiden'], show=False, 
                   save='_' + sub_name + '.pdf')        

    print('embedding with FA...')
    sc.tl.draw_graph(adata, init_pos='paga')

    sc.pl.draw_graph(adata, color=['timepoint', 'dose', 'leiden', 'batch', 'Subject ID', 'type'], show=False,
                     save='_' + sub_name + '.pdf')

    sc.pl.draw_graph(adata, color=adata.var.index.values, show=False, use_raw=True, 
                     save='_' + sub_name + '_markers.pdf')

    print('embedding with density plot...')


    sc.tl.embedding_density(adata, basis='draw_graph_fa', groupby='timepoint')
    sc.pl.embedding_density(adata, basis='draw_graph_fa', key='draw_graph_fa_density_timepoint', 
                            group=density_groups, show=False, 
                            save='_' + sub_name + '_timepoint_density.pdf')

    print('saving results...')
    adata.write(filename=os.path.join(h5path, 'EXP-21-DG3637_56_' + sub_name + '_sample3000.h5ad'), compression = 'gzip')
    adata.X = adata.raw.X[:,~drop_indicator]
    adata.write(filename=os.path.join(h5path, 'EXP-21-DG3637_56_' + sub_name + '_sample3000_raw.h5ad'), compression = 'gzip')
    print('done!')

subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
